# Running and MNIST dataset classifier through CUSTOM image endpoints
---
## Adding package to the classpath
First of all we need to add the main package to the classpath so that the notebook can load all the necessary libraries from konduit-serving into the Jupyter notebook kernel.

Classpaths can be considered similar to `site-packages` in the python ecosystem where each library that's to be imported to your code is loaded from.

In [1]:
%classpath add jar ../../konduit.jar

In [2]:
%%bash
less train.py

import tensorflow as tf

from keras.datasets import mnist


tensorflow_version = tf.__version__
print(tensorflow_version)

# Load data
train_data, test_data = mnist.load_data()
x_train, y_train = train_data
x_test, y_test = test_data

# Normalize
x_train = x_train / 255.0
x_test = x_test / 255.0


def get_model():
    inputs = tf.keras.layers.Input(shape=(28, 28), name="input_layer")
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(200, activation="relu")(x)
    x = tf.keras.layers.Dense(100, activation="relu")(x)
    x = tf.keras.layers.Dense(60, activation="relu")(x)
    x = tf.keras.layers.Dense(30, activation="relu")(x)
    outputs = tf.keras.layers.Dense(10, activation="softmax", name="output_layer")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer='sgd',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


def train(epochs=8):
    model = get_model()
    mode

In [3]:
%%bash
konduit serve -id keras-mnist -c keras.json -rwm -b

Starting konduit server...
Using classpath: /root/konduit/bin/../konduit.jar
INFO: Running command /root/miniconda/jre/bin/java -Dkonduit.logs.file.path=/root/.konduit-serving/command_logs/keras-mnist.log -Dlogback.configurationFile=/tmp/logback-run_command_b3fa9b25ed4c41f9.xml -jar /root/konduit/bin/../konduit.jar run --instances 1 -s inference -c keras.json -Dserving.id=keras-mnist
For server status, execute: 'konduit list'
For logs, execute: 'konduit logs keras-mnist'



In [4]:
%%bash
konduit list


Listing konduit servers...

 #   | ID                             | TYPE       | URL                  | PID     | STATUS     
 1   | keras-mnist                    | inference  | waiting...           | 1200    | Starting: Execute 'konduit logs keras-mnist' for more details... 




In [5]:
%%bash
konduit logs keras-mnist --lines 100

15:00:08.259 [vert.x-worker-thread-0] ERROR o.n.common.config.ND4JClassLoading - Cannot find class [org.canova.api.io.data.DoubleWritable] of provided class-loader.
15:00:08.310 [vert.x-worker-thread-0] ERROR o.n.common.config.ND4JClassLoading - Cannot find class [org.nd4j.linalg.jblas.JblasBackend] of provided class-loader.
15:00:08.310 [vert.x-worker-thread-0] ERROR o.n.common.config.ND4JClassLoading - Cannot find class [org.canova.api.io.data.DoubleWritable] of provided class-loader.
15:00:08.489 [vert.x-worker-thread-0] INFO  org.nd4j.nativeblas.NativeOpsHolder - Number of threads used for linear algebra: 8
15:00:08.490 [vert.x-worker-thread-0] WARN  o.n.l.c.nativecpu.CpuNDArrayFactory - *********************************** CPU Feature Check Warning ***********************************
15:00:08.490 [vert.x-worker-thread-0] WARN  o.n.l.c.nativecpu.CpuNDArrayFactory - Warning: Initializing ND4J with Generic x86 binary on a CPU with AVX/AVX2 support
15:00:08.490 [vert.x-worker-thread-0]

In [6]:
%%html
<img src="test-image.jpg" alt="title">

In [7]:
%%bash
konduit predict keras-mnist -it multipart 'image=@test-image.jpg'

{
  "output_layer" : [ [ 9.0376153E-7, 1.0595608E-8, 1.3115231E-5, 0.44657645, 6.748624E-12, 0.5524258, 1.848306E-7, 2.7652052E-9, 9.76023E-4, 7.5933513E-6 ] ],
  "prob" : 0.5524258017539978,
  "index" : 5,
  "label" : "5"
}



In [8]:
%%bash
konduit stop keras-mnist

Stopping konduit server 'keras-mnist'
Application 'keras-mnist' terminated with status 0

